In [159]:
# activate R magic
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


#**Prevendo a Ocorrência de Câncer**

Problema de Negócio: Previsão de Ocorrência de Câncer de Mama

[link text](http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29)

---

##**Etapa 1** - Coletando Dados

Os dados do câncer da mama incluem 569 observações de biópsias de câncer, cada um com 32 características (variáveis). 

#### Variável Target: **O diagnóstico é codificado como "M" para indicar maligno ou "B" para indicar benigno.** 

In [160]:
%%R
  dados <- read.csv("dataset.csv", stringsAsFactors = FALSE) # Indica a não considerar nenhuma variável numérica como sendo do tipo fator
  str(dados)

'data.frame':	569 obs. of  32 variables:
 $ id               : int  87139402 8910251 905520 868871 9012568 906539 925291 87880 862989 89827 ...
 $ diagnosis        : chr  "B" "B" "B" "B" ...
 $ radius_mean      : num  12.3 10.6 11 11.3 15.2 ...
 $ texture_mean     : num  12.4 18.9 16.8 13.4 13.2 ...
 $ perimeter_mean   : num  78.8 69.3 70.9 73 97.7 ...
 $ area_mean        : num  464 346 373 385 712 ...
 $ smoothness_mean  : num  0.1028 0.0969 0.1077 0.1164 0.0796 ...
 $ compactness_mean : num  0.0698 0.1147 0.078 0.1136 0.0693 ...
 $ concavity_mean   : num  0.0399 0.0639 0.0305 0.0464 0.0339 ...
 $ points_mean      : num  0.037 0.0264 0.0248 0.048 0.0266 ...
 $ symmetry_mean    : num  0.196 0.192 0.171 0.177 0.172 ...
 $ dimension_mean   : num  0.0595 0.0649 0.0634 0.0607 0.0554 ...
 $ radius_se        : num  0.236 0.451 0.197 0.338 0.178 ...
 $ texture_se       : num  0.666 1.197 1.387 1.343 0.412 ...
 $ perimeter_se     : num  1.67 3.43 1.34 1.85 1.34 ...
 $ area_se          : num  1

## ATENÇÃO! Temos escalas numéricas diferentes para algumas variáveis.


---

##**Etapa 2** - Pré-Processamento


Excluindo a coluna ID Independentemente do método de aprendizagem de máquina, deve sempre ser excluídas variáveis de ID. É uma variável que apresenta a identificação de cada registro.

Caso contrário, isso pode levar a resultados errados porque o ID 
pode ser usado para unicamente "prever" cada exemplo.  

Im modelo  que inclui um identificador pode sofrer de superajuste (overfitting), e será muito difícil usá-lo para generalizar outros dados.

In [0]:
%%R
  dados$id = NULL
  # Removendo a variável ID do conjunto de dados de análise

### Ajustando o label da variável alvo

In [163]:
%%R
  dados$diagnosis = sapply(dados$diagnosis,                                          # utilizar a função sapply(), selecionar a coluna "diagnosis" em dados, atualizando a mesma coluna
                           function(x){ ifelse (x =='M', 'Maligno', 'Benigno')})     # criar uma função ifelse se o valor for igual a M, é Maligno. Caso contrário, benigno.
  str(dados)

'data.frame':	569 obs. of  31 variables:
 $ diagnosis        : chr  "Benigno" "Benigno" "Benigno" "Benigno" ...
 $ radius_mean      : num  12.3 10.6 11 11.3 15.2 ...
 $ texture_mean     : num  12.4 18.9 16.8 13.4 13.2 ...
 $ perimeter_mean   : num  78.8 69.3 70.9 73 97.7 ...
 $ area_mean        : num  464 346 373 385 712 ...
 $ smoothness_mean  : num  0.1028 0.0969 0.1077 0.1164 0.0796 ...
 $ compactness_mean : num  0.0698 0.1147 0.078 0.1136 0.0693 ...
 $ concavity_mean   : num  0.0399 0.0639 0.0305 0.0464 0.0339 ...
 $ points_mean      : num  0.037 0.0264 0.0248 0.048 0.0266 ...
 $ symmetry_mean    : num  0.196 0.192 0.171 0.177 0.172 ...
 $ dimension_mean   : num  0.0595 0.0649 0.0634 0.0607 0.0554 ...
 $ radius_se        : num  0.236 0.451 0.197 0.338 0.178 ...
 $ texture_se       : num  0.666 1.197 1.387 1.343 0.412 ...
 $ perimeter_se     : num  1.67 3.43 1.34 1.85 1.34 ...
 $ area_se          : num  17.4 27.1 13.5 26.3 17.7 ...
 $ smoothness_se    : num  0.00805 0.00747 0.00516 

### Muitos classificadores requerem que as variáveis sejam do tipo fator

In [166]:
%%R 
  table(dados$diagnosis)
  # Criar a tabela de Cntingência da Variável Alvo com valores de "benigno" e "maligno"


Benigno Maligno 
    357     212 


### Fatorando a variável target diagnóstico 

In [0]:
%%R
  dados$diagnosis <- factor(dados$diagnosis, levels = c("Benigno", "Maligno"), labels = c("Benigno", "Maligno"))
  # a variável target que era caracter agora passa a ser fator

In [168]:
%%R
  str(dados$diagnosis)

 Factor w/ 2 levels "Benigno","Maligno": 1 1 1 1 1 1 1 2 1 1 ...


###Verificando a proporção das categorias de Câncer

In [169]:
%%R
  round(prop.table(table(dados$diagnosis)) * 100, digits = 1)


Benigno Maligno 
   62.7    37.3 


###Medidas de Tendência Central

Detectamos um problema de escala entre os dados, que então precisam ser normalizados. 

O cálculo de distância feito pelo kNN é dependente das medidas de escala nos dados de entrada.

In [170]:
%%R
  summary(dados[c("radius_mean", "area_mean", "smoothness_mean")])

  radius_mean      area_mean    smoothness_mean 
 Min.   : 6.98   Min.   : 144   Min.   :0.0526  
 1st Qu.:11.70   1st Qu.: 420   1st Qu.:0.0864  
 Median :13.37   Median : 551   Median :0.0959  
 Mean   :14.13   Mean   : 655   Mean   :0.0964  
 3rd Qu.:15.78   3rd Qu.: 783   3rd Qu.:0.1053  
 Max.   :28.11   Max.   :2501   Max.   :0.1634  


### Criando uma função de **Normalização** para converter os dados numéricos na mesma escala


#### Muitos algoritmos esperam receber os mesmos dados na mesma escala e com uma distribuição normal. Média igual a 0 e desvio-padrão igual a 1. 

#### Na maioria das vezes os dados não virão na mesma escala e nem com uma distribuição normal. Cabe então identificar e normalizar se necessário. 


In [0]:
%%R 
  normalizar <- function(x) { # "Padronização"
  return ((x - min(x)) / (max(x) - min(x))) # a função pega o valor "x" que rece e subtrai do valor mínimo.
}                                           # dividirá pelo valor máximo de "x" subtraído deo mínimo de "x"

#### Testando a função de normalização - **os resultados devem ser idênticos**

In [135]:
%%R
  normalizar(c(1, 2, 3, 4, 5)) # valores unitários coincidem com a dezena - confirma a normalização 
  

[1] 0.00 0.25 0.50 0.75 1.00


In [136]:
%%R
  normalizar(c(10, 20, 30, 40, 50)) # valores em dezena concidem com unitários - confirma a normalização 

[1] 0.00 0.25 0.50 0.75 1.00


###Normalizando os dados com lapply() 

#### lapply() percorrerá o conjunto "dados" da coluna 2 à 31, aplicando a função criada normalizar(). 

#### a saída sera um novo dataset chamado dados_norm com as variáveis numéricas em escala. 

In [0]:
%%R
  dados_norm <- as.data.frame(lapply(dados[2:31], normalizar))
  head(dados_norm)

---

##**Etapa 3** - Treinando o modelo com KNN

### Um dos algoritmos de Machine Learning mais simples que existem 

In [0]:
%%R
  # Carregando o pacote library class para rodar o KNN
  install.packages("class")
  library(class)
  # ?knn

### Criando dados de treino e dados de teste

In [0]:
%%R
  #notação de índice  
  dados_treino <- dados_norm[1:469, ]  # Selecionando 469 linhas e todas as suas colunas, aplicando no dataset "treino"
  dados_teste <- dados_norm[470:569, ] # Selecionando linhas restantes e todas as suas colunas, aplicando no dataset "teste"

### Criando Labels para os dados de treino e de teste

Labels são as palavras | valores | siglas usadas para aplicação da **variável target**.




In [175]:
%%R
  dados_treino_labels <- dados[1:469, 1] # Aplicando a identificação dos labels à PRIMEIRA COLUNHA e suas 469 linhas
  length(dados_treino_labels) # Retornar comprimento dos dados de Treino


[1] 469


In [176]:
 %%R 
  dados_teste_labels <- dados[470:569, 1] # Aplicando a identificação dos labels à PRIMEIRA COLUNHA e suas 100 linhas restantes
  length(dados_teste_labels) # Retornar comprimento dos dados de Teste

[1] 100


### Criando Modelo kNN

In [0]:
%%R
  modelo_knn_v1 <- knn(train = dados_treino,   # Chamar a função knn indicando dados de treino
                     test = dados_teste,       # indicar dados de teste 
                     cl = dados_treino_labels, # aplicar slicing de lables anterior
                     k = 21)                   # este k = 21 indica que o modelo observará 21 pontos de dados mais próximos de cada ponto de dado, com base na distância euclidiana

### A **função knn() retorna um objeto do tipo fator** com as previsões para cada exemplo no dataset de teste

In [143]:
%%R
  summary(modelo_knn_v1)

Benigno Maligno 
     63      37 


---

##**Etapa 4** - Avaliando e Interpretando o Modelo

In [0]:
%%R
  install.packages("gmodels")
  library(gmodels)

### Criando uma tabela cruzada dos dados previstos x dados atuais. 
### Usaremos amostra com 100 observações: length(dados_teste_labels)

In [180]:
%%R
  # Criando matriz de confusão 
  CrossTable(x = dados_teste_labels, # Selecionar labels de Teste
             y = modelo_knn_v1,      # Aplicar Movedelo V1 anteriormente criado
             prop.chisq = FALSE)     # Não imprimir estatísticas de Qui-Quadrado


 
   Cell Contents
|-------------------------|
|                       N |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  100 

 
                   | modelo_knn_v1 
dados_teste_labels |   Benigno |   Maligno | Row Total | 
-------------------|-----------|-----------|-----------|
           Benigno |        61 |         0 |        61 | 
                   |     1.000 |     0.000 |     0.610 | 
                   |     0.968 |     0.000 |           | 
                   |     0.610 |     0.000 |           | 
-------------------|-----------|-----------|-----------|
           Maligno |         2 |        37 |        39 | 
                   |     0.051 |     0.949 |     0.390 | 
                   |     0.032 |     1.000 |           | 
                   |     0.020 |     0.370 |           | 
-------------------|-----------|-----------|-----------|
      Column Total |        63 |        37

###**Interpretando os Resultados da Matriz de Confusão acima:**

In [0]:
# A tabela cruzada mostra 4 possíveis valores, que representam:

TP | FP
FN | TN

# Temos:
# Cenário 1: Célula Benigno (Observado) x Benigno (Previsto) - 61 casos - true positive 
# Cenário 2: Célula Maligno (Observado) x Benigno (Previsto) - 00 casos - false positive (o modelo errou)
# Cenário 3: Célula Benigno (Observado) x Maligno (Previsto) - 02 casos - false negative (o modelo errou) não conseguiu compreender tudo que existe de relação entre os dados
# Cenário 4: Célula Maligno (Observado) x Maligno (Previsto) - 37 casos - true negative 

# Lendo a Confusion Matrix (Perspectiva de ter ou não a doença):

# True Negative  = nosso modelo previu que a pessoa NÃO tinha a doença e os dados mostraram que realmente a pessoa NÃO tinha a doença
# False Positive = nosso modelo previu que a pessoa tinha a doença e os dados mostraram que NÃO, a pessoa tinha a doença
# False Negative = nosso modelo previu que a pessoa NÃO tinha a doença e os dados mostraram que SIM, a pessoa tinha a doença
# True Positive = nosso modelo previu que a pessoa tinha a doença e os dados mostraram que SIM, a pessoa tinha a doença

# ------------------------------------------------------------------------

# Erros podem ter consequências diferentes
# Falso Positivo - Erro Tipo I 
# Falso Negativo - Erro Tipo II

# Taxa de acerto deste  Modelo: 98% (acertou 98 em 100)

---

##**Etapa 5** - Otimizando a Performance do Modelo

### Usando a função scale() para padronizar na mesma escala do z-score 

In [0]:
%%R
  #?scale() a função faz uma escala e centralização dos objetos. scale(conjuntod de dados)
  dados_z <- as.data.frame(scale(dados[-1])) # -1 é para excluir justamente a coluna que temos a variável alvo e aplicar em dados_Z

### Confirmando transformação realizada com sucesso


In [182]:
%%R
  summary(dados_z$area_mean)
  # temos a média igual a 0

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 -1.453  -0.667  -0.295   0.000   0.363   5.246 


### Fazendo uma nova divisão em dados de Treino e de Teste - **Novamente**

In [0]:
%%R 
 # Dados de Treino
 dados_treino <- dados_z[1:469, ]

In [0]:
%%R
  # Dados de Teste
  dados_teste <- dados_z[470:569, ]

### Criando Labels para os dados de treino e de teste - **Novamente**


In [186]:
%%R
  dados_treino_labels <- dados[1:469, 1] # Aplicando a identificação dos labels à PRIMEIRA COLUNHA e suas 469 linhas
  length(dados_treino_labels) # Retornar comprimento dos dados de Treino

[1] 469


In [187]:
 %%R 
  dados_teste_labels <- dados[470:569, 1] # Aplicando a identificação dos labels à PRIMEIRA COLUNHA e suas 100 linhas restantes
  length(dados_teste_labels) # Retornar comprimento dos dados de Teste

[1] 100


### Reclassificando como modelo kNN Versão 2

In [0]:
%%R
  modelo_knn_v2 <- knn(train = dados_treino,   # Chamar a função knn 2 indicandos os novos dados de treino
                     test = dados_teste,       # indicar novos dados de teste
                     cl = dados_treino_labels, # Aplicar slicing de labels anteriores
                     k = 21)                   # este k = 21 indica que o modelo observará 21 pontos de dados mais próximos de cada ponto de dado, com base na distância euclidiana
  

### Criando uma tabela cruzada dos dados previstos x dados atuais


In [190]:
%%R
  CrossTable(x = dados_teste_labels, y = modelo_knn_v2, prop.chisq = FALSE)
  # É possível notar que a quantidade de erros aumentou


 
   Cell Contents
|-------------------------|
|                       N |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  100 

 
                   | modelo_knn_v2 
dados_teste_labels |   Benigno |   Maligno | Row Total | 
-------------------|-----------|-----------|-----------|
           Benigno |        61 |         0 |        61 | 
                   |     1.000 |     0.000 |     0.610 | 
                   |     0.924 |     0.000 |           | 
                   |     0.610 |     0.000 |           | 
-------------------|-----------|-----------|-----------|
           Maligno |         5 |        34 |        39 | 
                   |     0.128 |     0.872 |     0.390 | 
                   |     0.076 |     1.000 |           | 
                   |     0.050 |     0.340 |           | 
-------------------|-----------|-----------|-----------|
      Column Total |        66 |        34

### Experimentar diferentes valores para *K*

---

##**Etapa 6** - Construindo um Modelo com Algoritmo **Support Vector Machine (SVM)**

Trabalhar com dados não linearmente separáveis. 

Pode ser aplicado em **Regressão** e **Classificação**

### Preparar dataset

In [0]:
%%R
set.seed(40) 

In [193]:
%%R
  dados <- read.csv("dataset.csv", stringsAsFactors = FALSE) # Carregar novamente o conjunto de dados
  dados$id = NULL # Remover a coluna ID
  str(dados)

'data.frame':	569 obs. of  31 variables:
 $ diagnosis        : chr  "B" "B" "B" "B" ...
 $ radius_mean      : num  12.3 10.6 11 11.3 15.2 ...
 $ texture_mean     : num  12.4 18.9 16.8 13.4 13.2 ...
 $ perimeter_mean   : num  78.8 69.3 70.9 73 97.7 ...
 $ area_mean        : num  464 346 373 385 712 ...
 $ smoothness_mean  : num  0.1028 0.0969 0.1077 0.1164 0.0796 ...
 $ compactness_mean : num  0.0698 0.1147 0.078 0.1136 0.0693 ...
 $ concavity_mean   : num  0.0399 0.0639 0.0305 0.0464 0.0339 ...
 $ points_mean      : num  0.037 0.0264 0.0248 0.048 0.0266 ...
 $ symmetry_mean    : num  0.196 0.192 0.171 0.177 0.172 ...
 $ dimension_mean   : num  0.0595 0.0649 0.0634 0.0607 0.0554 ...
 $ radius_se        : num  0.236 0.451 0.197 0.338 0.178 ...
 $ texture_se       : num  0.666 1.197 1.387 1.343 0.412 ...
 $ perimeter_se     : num  1.67 3.43 1.34 1.85 1.34 ...
 $ area_se          : num  17.4 27.1 13.5 26.3 17.7 ...
 $ smoothness_se    : num  0.00805 0.00747 0.00516 0.01127 0.00501 ...
 $ c

### Criando uma coluna de Índice para separar os dados de Treino e de Teste

In [195]:
%%R
  dados[,'index'] <- ifelse(runif(nrow(dados)) < 0.8,1,0)
  str(dados)

'data.frame':	569 obs. of  32 variables:
 $ diagnosis        : chr  "B" "B" "B" "B" ...
 $ radius_mean      : num  12.3 10.6 11 11.3 15.2 ...
 $ texture_mean     : num  12.4 18.9 16.8 13.4 13.2 ...
 $ perimeter_mean   : num  78.8 69.3 70.9 73 97.7 ...
 $ area_mean        : num  464 346 373 385 712 ...
 $ smoothness_mean  : num  0.1028 0.0969 0.1077 0.1164 0.0796 ...
 $ compactness_mean : num  0.0698 0.1147 0.078 0.1136 0.0693 ...
 $ concavity_mean   : num  0.0399 0.0639 0.0305 0.0464 0.0339 ...
 $ points_mean      : num  0.037 0.0264 0.0248 0.048 0.0266 ...
 $ symmetry_mean    : num  0.196 0.192 0.171 0.177 0.172 ...
 $ dimension_mean   : num  0.0595 0.0649 0.0634 0.0607 0.0554 ...
 $ radius_se        : num  0.236 0.451 0.197 0.338 0.178 ...
 $ texture_se       : num  0.666 1.197 1.387 1.343 0.412 ...
 $ perimeter_se     : num  1.67 3.43 1.34 1.85 1.34 ...
 $ area_se          : num  17.4 27.1 13.5 26.3 17.7 ...
 $ smoothness_se    : num  0.00805 0.00747 0.00516 0.01127 0.00501 ...
 $ c

#### Dados de Treino com chamada pelo Índice = 1

In [0]:
%%R
  trainset <- dados[dados$index==1,] # Obter dados de treino para o objeto trainset

#### Dados de Teste com chamada pelo Índice = 0




In [0]:
%%R
  testset <- dados[dados$index==0,] # Obter dados de teste para o objeto testset

####Obter o Index de treino

In [0]:
%%R
trainColNum <- grep("index", names(trainset)) # Obter o índice apenas do conjunto de dados de treino

### Remover o índice dos datasets

In [0]:
%%R
  trainset <- trainset[,-trainColNum]

In [0]:
%%R
  testset <- testset[, -trainColNum]

###Obter Índice de coluna da variável target no conjunto de dados

In [0]:
%%R
typeColNum <- grep("diag", names(dados)) # grep() pesquisa pelo nome de variável que começa com "diag"

---

In [0]:
%%R
 install.packages("e1071") 
 library(e1071)

In [0]:
#?svw

## Criar Modelo

### Ajustamos Kernel para radial, já que este conjunto não tem um plano linear que pode ser desenhado


In [0]:
%%R
  modelo_svm_v1 <- svm(diagnosis ~ .,            # passar variável target relacionada com todas as outras
                     data = trainset,            # indicar dataset de treino
                     type = 'C-classification',  # ajustar tipo para modelo de classificação
                     kernel = 'radial')          # kernel é a base do algoritmo svm

#Previsões

### Previsões aplicadas no dataset de TREINO

In [0]:
%%R
  pred_train <- predict(modelo_svm_v1, trainset) 

### Calcular o percentual de previssões acertadas com dataset de treino

In [209]:
%%R
  mean_pred_train <- mean(pred_train == trainset$diagnosis, n = 6)  
  mean_pred_train 
  # com dados de históricos de treino o modelo acertou 98.47 das vezes, praticamente o mesmo com kNN
  # mudar de algoritmo não foi tão positivo, provavelmente estamos no limite do dataset

[1] 0.988688


### Previsões nos dados de Teste

In [0]:
%%R
  pred_test <- predict(modelo_svm_v1, testset) 

### Percentual de previsões corretas com dataset de teste

In [211]:
%%R
  mean(pred_test == testset$diagnosis)  

[1] 0.96063


### Matriz de Confusão 

In [212]:
%%R 
  table(pred_test, testset$diagnosis)

         
pred_test  B  M
        B 82  3
        M  2 40


---

##**Etapa 7** - Construindo um Modelo com Algoritmo Random Forest

### Quando algumas variáveis não tem um nível de relevância alto, elas afetam a Performance do modelo. 

### Criando o modelo

In [0]:
%%R
  install.packages("rpart")
  library(rpart)

In [0]:
%%R
  modelo_rf_v1 = rpart(diagnosis ~ .,                       # Chamar função rpart() e passar a variável target relacionado com as demais preditoras
                       data = trainset,                     # Indicar qual é o dataset
                       control = rpart.control(cp = .0005)) # Ajustar rpart.controlo com nível de precisão .0005, o nívlel das folhas

###Previsões nos dados de teste

In [0]:
%%R
  tree_pred = predict(modelo_rf_v1, testset, type='class')

### Calcular percentual de acertos com dados de Teste

#### Tivemos uma performance pior do que com a aplicação do kNN

In [215]:
%%R
  mean(tree_pred==testset$diagnosis) 


[1] 0.92126


### Matriz de Confusão 


In [216]:
%%R
  table(tree_pred, testset$diagnosis)

         
tree_pred  B  M
        B 79  5
        M  5 38
